In [55]:
import os
import pandas as pd
from pathlib import Path
import openai

# API Key Setup using Doppler
openai_key = os.environ.get("OPENAI_API_KEY")
if not openai_key:
    raise RuntimeError("OPENAI_API_KEY not found.")

openai.api_key = openai_key

book_df = pd.read_csv("2024BookList.csv")

#Creating a filtered version of the BookList
filtered_df = book_df[['MonthRead', 'MonthName', 'Title', 'Tags']].dropna()

filtered_df




,MonthRead,MonthName,Title,Tags
1,1,January,Atalanta,"Fiction, Fairy Tales; Folk Tales; Legends & My..."
2,1,January,Ariadne: A Novel,"Fiction, Literary, Fairy Tales; Folk Tales; Le..."
4,1,January,The Last Olympian,"Juvenile Fiction, General, Action & Adventure,..."
5,1,January,The Lightning Thief,"Juvenile Fiction, Media Tie-In, Action & Adven..."
6,1,January,The Sea of Monsters,"Cooking, Courses & Dishes, Confectionery, Juve..."
9,1,January,The Chalice of the Gods,"Fantasy, young adult, Adventure, Childrens"
10,1,January,The Battle of the Labyrinth,"Juvenile Fiction, Action & Adventure, General,..."
11,2,February,Loveless,"Young Adult Fiction, Lgbtq, Social Themes, Emo..."
12,2,February,Nick and Charlie,"Young Adult Fiction, Coming of Age, Lgbtq, Peo..."
15,2,February,Heartstopper Volume 4,"Juvenile Fiction, Love & Romance, Social Theme..."


In [56]:



# Create book blurbs of tags for each month
month_blobs = {}
for month, group in filtered_df.groupby('MonthName'):
    blob = "\n".join(
        f"- {row['Title']}: {str(row['Tags'])[:100]}"
        for _, row in group.iterrows()
    )
    month_blobs[month] = blob

# Create OpenAI-style prompts
openai_prompts = []
for month, blob in month_blobs.items():
    month_tags = filtered_df[filtered_df['MonthName'] == month]['Tags'].dropna()
    top_tags = month_tags.value_counts().head(3).index.tolist()
    tags_str = ', '.join(top_tags)
    # Get the corresponding MonthRead value (assumes all rows for a month have the same MonthRead)
    month_read = filtered_df[filtered_df['MonthName'] == month]['MonthRead'].iloc[0]
    daylist_prompt = f"""Here are the books I read in {month} 2024:\n\n{blob}\n\nBased on this, give me a Spotify Daylist-style mood name. It should be 3 to 4 words followed by the month, lowercase, vibe-based, and playful.\nExample: 'melancholy vampire September', 'satire audiobook hour February', 'dreamcore whimsical deligh May'\n\n Please provide the mood name for {month} 2024"""
    openai_prompts.append({
        "MonthName": month,
        "MonthRead": month_read,
        "Daylist_Prompt": daylist_prompt,
    })

# Store prompt df
month_df = pd.DataFrame(openai_prompts, columns=["MonthName", "MonthRead", "Daylist_Prompt"])
month_df = month_df.sort_values(by='MonthRead').reset_index(drop=True)
# Add a column for books read count in each month
month_df["Books_Read"] = month_df["MonthName"].apply(
    lambda month: filtered_df[filtered_df['MonthName'] == month]['Title'].nunique()
)

month_df




,MonthName,MonthRead,Daylist_Prompt,Books_Read
0,January,1,Here are the books I read in January 2024:\n\n...,7
1,February,2,Here are the books I read in February 2024:\n\...,4
2,March,3,Here are the books I read in March 2024:\n\n- ...,3
3,April,4,Here are the books I read in April 2024:\n\n- ...,6
4,May,5,Here are the books I read in May 2024:\n\n- Ho...,10
5,June,6,Here are the books I read in June 2024:\n\n- H...,3
6,July,7,Here are the books I read in July 2024:\n\n- A...,8
7,August,8,Here are the books I read in August 2024:\n\n-...,4
8,September,9,Here are the books I read in September 2024:\n...,2
9,October,10,Here are the books I read in October 2024:\n\n...,2


In [57]:

# Function to get mood name from prompt using OpenAI, ChatGPT used to come up with a role 
def MonthReview(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a creative assistant for Spotify-style daylist mood naming and recommending books."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message["content"].strip()
    except Exception as e:
        return f"Error: {str(e)}"


# Apply to each prompt and get Daylist mood names
month_df["Daylist_Name"] = month_df["Daylist_Prompt"].apply(MonthReview)





In [58]:
for index, row in month_df.iterrows():
    print("You had a {} in {} 2024 with {} books read.".format(
        row["Daylist_Name"],
        row["MonthName"],
        row["Books_Read"],
    ))

You had a mythical adventure whimsy january in January 2024 with 7 books read.
You had a lovestruck romance vibes february in February 2024 with 4 books read.
You had a dysfunctional drama spotlight march in March 2024 with 3 books read.
You had a adventurous robot dreams april in April 2024 with 6 books read.
You had a mythical literary journey may in May 2024 with 10 books read.
You had a mythical storytelling adventure june in June 2024 with 3 books read.
You had a dark fantasy saga July in July 2024 with 8 books read.
You had a philosophical dreamer august in August 2024 with 4 books read.
You had a crazy memoir vibes september in September 2024 with 2 books read.
You had a mythical underwater dream october in October 2024 with 2 books read.
You had a bisexual literary kitchen november in November 2024 with 3 books read.
You had a psychological introspection december in December 2024 with 3 books read.
